In [1]:
#import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats

In [2]:
filelist = ['EIA930_BALANCE_2023_Jan_Jun.csv', 'EIA930_BALANCE_2023_Jul_Dec.csv','EIA930_BALANCE_2022_Jan_Jun.csv', 'EIA930_BALANCE_2022_Jul_Dec.csv']
data = pd.read_csv('EIA930_BALANCE_2023_Jan_Jun.csv', parse_dates=['Data Date'])
data = pd.concat(
    map(pd.read_csv, filelist), ignore_index=True)
data['Data Date']= pd.to_datetime(data['Data Date'])
data.index.rename('index',inplace=True)

C:\Users\suyas\AppData\Local\Temp\ipykernel_21880\4280652761.py:2: DtypeWarning: Columns (5,6,7,8,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('EIA930_BALANCE_2023_Jan_Jun.csv', parse_dates=['Data Date'])
C:\Users\suyas\AppData\Local\Temp\ipykernel_21880\4280652761.py:3: DtypeWarning: Columns (5,6,7,8,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat(
C:\Users\suyas\AppData\Local\Temp\ipykernel_21880\4280652761.py:3: DtypeWarning: Columns (19,27,30,31,32,36) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.concat(
C:\Users\suyas\AppData\Local\Temp\ipykernel_21880\4280652761.py:3: DtypeWarning: Columns (5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315190 entries, 0 to 315189
Data columns (total 42 columns):
 #   Column                                                             Non-Null Count   Dtype         
---  ------                                                             --------------   -----         
 0   Balancing Authority                                                315190 non-null  object        
 1   Data Date                                                          315190 non-null  datetime64[ns]
 2   Hour Number                                                        315190 non-null  int64         
 3   Local Time at End of Hour                                          315190 non-null  object        
 4   UTC Time at End of Hour                                            315190 non-null  object        
 5   Demand Forecast (MW)                                               269091 non-null  object        
 6   Demand (MW)                                         

Hour (wrt to local time), Date, Month, Year, Day of week, Day of year, Week of year, quarter.
Balancing Authority/Region
Non Renewable - Coal, Natural Gas, Petroleum
Renewable - Hydro, Solar, Wind, Nuclear
Other sources, unknown sources?

In [3]:
#deleting non-useful columns
non_useful = []
for col in data:
    if col[-1] == ')':
        non_useful.append(col)
non_useful.append('Local Time at End of Hour')
non_useful.append('UTC Time at End of Hour')
non_useful.append('Net Generation (MW) from Other Fuel Sources')
non_useful.append('Net Generation (MW) from Unknown Fuel Sources')
non_useful.append('Balancing Authority')
data = data.drop(non_useful,axis=1)

In [4]:
#generating features from date object
data['dayofmonth'] = data['Data Date'].dt.day
data['month'] = data['Data Date'].dt.month
data['year'] = data['Data Date'].dt.year
data['dayofweek'] = data['Data Date'].dt.dayofweek
data['dayofyear'] = data['Data Date'].dt.dayofyear
data['weekofyear'] = data['Data Date'].dt.isocalendar().week
data['quarter'] = data['Data Date'].dt.quarter

In [5]:
for col in data:
    if col.startswith('Net'):
        data[col] = data[col].str.replace(',','')
        data[col] = data[col].fillna(0).astype('int')
        data[col] = data[col].astype('int')
data.drop('Data Date',axis=1,inplace=True)

In [6]:
s1 = data['Net Generation (MW) from Hydropower and Pumped Storage']
s2 = data['Net Generation (MW) from Nuclear']
s3 = data['Net Generation (MW) from Solar']
s4 = data['Net Generation (MW) from Wind']

data['renewable'] = s1.add(s2).add(s3).add(s4)
data.drop(s1.name,axis=1,inplace=True)
data.drop(s2.name,axis=1,inplace=True)
data.drop(s3.name,axis=1,inplace=True)
data.drop(s4.name,axis=1,inplace=True)

In [7]:
s5 = data['Net Generation (MW) from Coal']
s6 = data['Net Generation (MW) from Natural Gas']
s7 = data['Net Generation (MW) from All Petroleum Products']

data['non-renewable'] = s5.add(s6).add(s7)
data.drop(s5.name,axis=1,inplace=True)
data.drop(s6.name,axis=1,inplace=True)
data.drop(s7.name,axis=1,inplace=True)

In [25]:
def normalize(col):
    data[col] = (data[col]-data[col].mean())/data[col].std().astype('float')

In [8]:
from sklearn.preprocessing import OneHotEncoder
cat_cols = [col for col in data if data[col].dtype=='object']
oh_encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
encoded_cols = pd.DataFrame(oh_encoder.fit_transform(data[cat_cols]))
encoded_cols.index = data.index
encoded_cols.columns = oh_encoder.get_feature_names_out(cat_cols)
dropped_data = data.drop(cat_cols,axis=1)
encoded_data = pd.concat([dropped_data,encoded_cols], axis=1)

In [27]:
data

,Hour Number,Region,dayofmonth,month,year,dayofweek,dayofyear,weekofyear,quarter,renewable,non-renewable
index,,,,,,,,,,,
0,1,MIDW,1,1,2023,6,1,52,1,75,2623
1,2,MIDW,1,1,2023,6,1,52,1,48,2600
2,3,MIDW,1,1,2023,6,1,52,1,15,2589
3,4,MIDW,1,1,2023,6,1,52,1,31,2593
4,5,MIDW,1,1,2023,6,1,52,1,80,2587
...,...,...,...,...,...,...,...,...,...,...,...
315185,20,CAR,30,7,2023,6,211,30,3,133,0
315186,21,CAR,30,7,2023,6,211,30,3,131,0
315187,22,CAR,30,7,2023,6,211,30,3,4,0


In [9]:
encoded_data

,Hour Number,dayofmonth,month,year,dayofweek,dayofyear,weekofyear,quarter,renewable,non-renewable,...,Region_FLA,Region_MIDA,Region_MIDW,Region_NE,Region_NW,Region_NY,Region_SE,Region_SW,Region_TEN,Region_TEX
index,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,2023,6,1,52,1,75,2623,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,1,2023,6,1,52,1,48,2600,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,1,1,2023,6,1,52,1,15,2589,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,1,1,2023,6,1,52,1,31,2593,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,1,2023,6,1,52,1,80,2587,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864136,20,31,12,2022,5,365,52,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
864137,21,31,12,2022,5,365,52,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
864138,22,31,12,2022,5,365,52,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
data.to_csv('data.csv')
encoded_data.to_csv('encoded_data.csv')